In [9]:
from dask.distributed import Client
import pandas as pd
import numpy as np
import time
import random
import dask

client = Client('129.82.208.160:8786')

def costly_simulation(list_param):
    time.sleep(random.random())
    return sum(list_param)

input_params = pd.DataFrame(np.random.random(size=(500, 4)),
                            columns=['param_a', 'param_b', 'param_c', 'param_d'])
input_params.head()

,param_a,param_b,param_c,param_d
0,0.686577,0.553213,0.210289,0.383761
1,0.171862,0.467209,0.103218,0.037029
2,0.652652,0.477734,0.492038,0.860714
3,0.653581,0.902217,0.906885,0.411982
4,0.664808,0.559564,0.980906,0.827694


In [7]:
results = []
for parameters in input_params.values[:10]:
    result = costly_simulation(parameters)
    results.append(result)
results

[1.4921727341728723,
 2.2360011507862607,
 1.8553111059353138,
 2.3749521658338373,
 1.5409643659665102,
 1.2277105874163925,
 1.0428286624611025,
 2.2929159366566,
 2.015045968219201,
 2.38567531066508]

In [13]:
%%time

lazy_results = []
for parameters in input_params.values[:10]:
    lazy_result = dask.delayed(costly_simulation)(parameters)
    lazy_results.append(lazy_result)

lazy_results[0]

CPU times: user 2.22 ms, sys: 122 µs, total: 2.34 ms
Wall time: 1.97 ms


Delayed('costly_simulation-e93fec89-620e-4d22-b05a-df20141b1078')

In [14]:
%time dask.compute(*lazy_results)

CPU times: user 19.9 ms, sys: 1.48 ms, total: 21.4 ms
Wall time: 1.15 s


(1.83383996001203,
 0.7793175885078301,
 2.4831376858764886,
 2.874665545072595,
 3.032972302002523,
 0.6973517718566685,
 1.6805098314518427,
 2.900002616266889,
 2.59840802162854,
 3.1003504498272068)

In [16]:
lazy_results = []

for parameters in input_params.values:
    lazy_result = dask.delayed(costly_simulation)(parameters)
    lazy_results.append(lazy_result)

futures = dask.persist(*lazy_results)  # trigger computation in the background
results = dask.compute(*futures)
results[:5]

(1.83383996001203,
 0.7793175885078301,
 2.4831376858764886,
 2.874665545072595,
 3.032972302002523)

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
